In [ ]:
import pandas as pd
import re
from tqdm.notebook import tqdm

In [ ]:
df_bond_ng = pd.read_csv('/content/drive/MyDrive/BOK_금리예측_비공개/data/bond_reports_all_ngramize.csv')
df_naver_news = pd.read_csv('/content/drive/MyDrive/BOK_금리예측/naver_news_ngramize_0.csv')
df_min_ng = pd.read_csv('/content/drive/MyDrive/BOK_금리예측_비공개/data/minutes_df_ngramize.csv')
df_call_rate = pd.read_csv('/content/drive/MyDrive/BOK_금리예측_비공개/data/callrate_final.csv')

In [ ]:
df_bond_ng['date_prep'] = df_bond_ng['date'].apply(lambda x : str(x)[:10])
df_bond_ng[['date_prep', 'ngram_prep']]

,date_prep,ngram_prep
0,2008-04-01,"[['신용경색/NNG;해소/NNG'], ['경기/NNG;침체/NNG;피하/VV'],..."
1,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG..."
2,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG..."
3,2008-04-21,"[['경기/NNG;바닥/NNG'], ['인하/NNG;영향/NNG;줄/VV', '인플..."
4,2008-04-28,"[['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격..."
...,...,...
3980,2022-11-14,"[['금리/NNG;하락/NNG', '하락/NNG;전환/NNG'], ['물가/NNG;..."
3981,2022-11-14,"[['유동성/NNG;확보/NNG'], ['수요/NNG;부진/NNG'], ['현금성자..."
3982,2022-11-15,"[['통화정책/NNG;긴축/NNG', '긴축/NNG;부담/NNG'], ['경기/NN..."
3983,2022-11-15,"[['현금성자산/NNG;감소/NNG', '가격/NNG;하락/NNG', '경기/NNG..."


In [ ]:
df_naver_news

,media,date,url,content_len,ngram_prep
0,연합뉴스,20080401,https://n.news.naver.com/mnews/article/001/000...,511,NaN
1,연합뉴스,20080401,https://n.news.naver.com/mnews/article/001/000...,848,"[['상환/NNG;부담/NNG;줄/VV'], ['금리/NNG;낮/VA']]"
2,연합뉴스,20080401,https://n.news.naver.com/mnews/article/001/000...,1499,"[['높/VA;감소/NNG'], ['원화/NNG;대출/NNG;증가/NNG'], ['..."
3,연합뉴스,20080401,https://n.news.naver.com/mnews/article/001/000...,259,NaN
4,연합뉴스,20080401,https://n.news.naver.com/mnews/article/001/000...,540,"[['기대/NNG;무산/NNG'], ['불안/NNG;증폭/NNG'], ['금리/NN..."
...,...,...,...,...,...
305824,이데일리,20221115,http://www.edaily.co.kr/news/newspath.asp?news...,1270,"[['위험/NNG;회피/NNG'], ['fed/NNG;매파/NNG;통화/NNG;긴축..."
305825,이데일리,20221115,http://www.edaily.co.kr/news/newspath.asp?news...,1068,"[['수신/NNG;금리/NNG;오르/VV', '금리/NNG;인상/NNG'], ['금..."
305826,이데일리,20221115,http://www.edaily.co.kr/news/newspath.asp?news...,1963,"[['crs/NNG;확대/NNG', '원달러/NNG;상승/NNG'], ['금리/NN..."
305827,이데일리,20221115,http://www.edaily.co.kr/news/newspath.asp?news...,1358,"[['금리/NNG;인상/NNG;지정학/NNG;위험/NNG', '불확실성/NNG;금리..."


In [ ]:
df_naver_news = df_naver_news.dropna(subset=['ngram_prep'])
df_naver_news['date_prep'] = df_naver_news['date'].apply(lambda x : f'{str(x)[:4]}-{str(x)[4:6]}-{str(x)[6:]}')
df_naver_news[['date_prep', 'ngram_prep']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,date_prep,ngram_prep
1,2008-04-01,"[['상환/NNG;부담/NNG;줄/VV'], ['금리/NNG;낮/VA']]"
2,2008-04-01,"[['높/VA;감소/NNG'], ['원화/NNG;대출/NNG;증가/NNG'], ['..."
4,2008-04-01,"[['기대/NNG;무산/NNG'], ['불안/NNG;증폭/NNG'], ['금리/NN..."
5,2008-04-01,"[['기업/NNG;자금사정/NNG;악화/NNG'], ['자금사정/NNG;좋/VA;않..."
7,2008-04-01,"[['금융시장/NNG;불안/NNG'], ['외국인/NNG;채권투자/NNG;급증/NN..."
...,...,...
305824,2022-11-15,"[['위험/NNG;회피/NNG'], ['fed/NNG;매파/NNG;통화/NNG;긴축..."
305825,2022-11-15,"[['수신/NNG;금리/NNG;오르/VV', '금리/NNG;인상/NNG'], ['금..."
305826,2022-11-15,"[['crs/NNG;확대/NNG', '원달러/NNG;상승/NNG'], ['금리/NN..."
305827,2022-11-15,"[['금리/NNG;인상/NNG;지정학/NNG;위험/NNG', '불확실성/NNG;금리..."


In [ ]:
df_min_ng.columns = ['date_prep', 'ngram_prep']
df_min_ng

,date_prep,ngram_prep
0,2022-10-27,"[['유동성/NNG;위축/NNG', '한시적/VAX;완화/NNG', '금리/NNG;..."
1,2022-10-12,"[['금리/NNG;상승/NNG'], ['재정/NNG;건전성/NNG;강화/NNG'],..."
2,2022-09-22,"[['금리/NNG;인상/NNG'], ['금리/NNG;인상/NNG;금융안정/NNG',..."
3,2022-09-08,"[['경기/NNG;하방/NNG;위험/NNG;증대/NNG', '물가/NNG;상승/NN..."
4,2022-08-25,"[['인플레이션/NNG;압력/NNG'], ['물가/NNG;확산/NNG', '지수/N..."
...,...,...
295,2008-05-08,"[['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NN..."
296,2008-04-10,"[['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/N..."
297,2008-03-20,"[['통화/NNG;안정/NNG'], ['물가/NNG;오름/NNG;확대/NNG', '..."
298,2008-03-07,"[['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/N..."


In [ ]:
# 컬럼명 통일 후 하나의 파일로 합치기
df_total = pd.concat([df_bond_ng[['date_prep', 'ngram_prep']], df_naver_news[['date_prep', 'ngram_prep']], df_min_ng], axis=0)
df_total = df_total.reset_index(drop=True)
df_total

,date_prep,ngram_prep
0,2008-04-01,"[['신용경색/NNG;해소/NNG'], ['경기/NNG;침체/NNG;피하/VV'],..."
1,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG..."
2,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG..."
3,2008-04-21,"[['경기/NNG;바닥/NNG'], ['인하/NNG;영향/NNG;줄/VV', '인플..."
4,2008-04-28,"[['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격..."
...,...,...
284001,2008-05-08,"[['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NN..."
284002,2008-04-10,"[['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/N..."
284003,2008-03-20,"[['통화/NNG;안정/NNG'], ['물가/NNG;오름/NNG;확대/NNG', '..."
284004,2008-03-07,"[['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/N..."


In [ ]:
df_total['ngram_word'] = ''
for idx in tqdm(df_total.index) :
    if str(df_total.loc[idx, 'ngram_prep']) != 'nan' :
        # 문자열 형식으로 저장한 list 복구 식
        result = []
        for ngm in re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣,.\/; ]', '', df_total.loc[idx, 'ngram_prep']).split(',') :
            temp = [ngm_word.strip() for ngm_word in ngm.split(';')]
            # print(temp)
            result.append(temp)
        # 문장별로 나눠진 ngram을 문서단위로 합쳐주기
        ngram_file = []
        for res in result:
            ngram_file.append(';'.join(res))
        df_total.loc[idx, 'ngram_word'] = str(ngram_file)

  0%|          | 0/284006 [00:00<?, ?it/s]

In [ ]:
df_total

,date_prep,ngram_prep,ngram_word
0,2008-04-01,"[['신용경색/NNG;해소/NNG'], ['경기/NNG;침체/NNG;피하/VV'],...","['신용경색/NNG;해소/NNG', '경기/NNG;침체/NNG;피하/VV', '실물..."
1,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG...","['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG'..."
2,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG...","['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG'..."
3,2008-04-21,"[['경기/NNG;바닥/NNG'], ['인하/NNG;영향/NNG;줄/VV', '인플...","['경기/NNG;바닥/NNG', '인하/NNG;영향/NNG;줄/VV', '인플레이션..."
4,2008-04-28,"[['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격...","['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격/..."
...,...,...,...
284001,2008-05-08,"[['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NN...","['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NNG..."
284002,2008-04-10,"[['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/N...","['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/NN..."
284003,2008-03-20,"[['통화/NNG;안정/NNG'], ['물가/NNG;오름/NNG;확대/NNG', '...","['통화/NNG;안정/NNG', '물가/NNG;오름/NNG;확대/NNG', '물가/..."
284004,2008-03-07,"[['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/N...","['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/NN..."


In [ ]:
# ngram 문서단위로 조합 합쳐진거 확인
df_total.loc[0,'ngram_prep']

"[['신용경색/NNG;해소/NNG'], ['경기/NNG;침체/NNG;피하/VV'], ['실물/NNG;경기/NNG;침체/NNG', '금리/NNG;하락/NNG'], ['금리/NNG;우호적/VAX', '위험/NNG;경계/NNG'], ['신흥국/NNG;신용위험/NNG', '부동산/NNG;위험/NNG'], ['정책/NNG;위험/NNG', '금리/NNG;인하/NNG'], ['변동성/NNG;축소/NNG'], ['금리/NNG;인하/NNG'], ['인플레이션/NNG;우려/NNG', '금리/NNG;인하/NNG'], ['유가/NNG;원자재/NNG;가격/NNG;상승/NNG', '경상/NNG;수지/NNG;적자/NNG;확대/NNG', '글로벌/NNG;신용/NNG;불안/NNG', '외국인/NNG;금리/NNG;상승/NNG', '유동성/NNG;여건/NNG;악화/NNG', '원달러/NNG;급등/NNG'], ['crs/NNG;금리/NNG;급락/NNG', '금리/NNG;급락/NNG;상승/NNG'], ['금리/NNG;인하/NNG', '주문/NNG;하락/NNG', '금리/NNG;심하/VA'], ['금리/NNG;인하/NNG', '인하/NNG;반대/NNG'], ['cd/NNG;금리/NNG;상승/NNG', '변동성/NNG;확대/NNG', '금리/NNG;하락/NNG', '하락/NNG;추세/NNG'], ['붕괴/NNG;위험/NNG'], ['침체/NNG;피하/VV;없/VA', '부동산/NNG;침체/NNG', '부채/NNG;부담/NNG', '경기/NNG;침체/NNG'], ['글로벌/NNG;경기/NNG;둔화/NNG'], ['위험/NNG;없/VA;않/VX'], ['gdp/NNG;성장률/NNG;둔화/NNG'], ['내수/NNG;늘/VV'], ['부정적/VAX;여파/NNG;우려/NNG'], ['정책/NNG;대응/NNG;한계/NNG', '본질/NNG;위험/NNG'], ['금리/NNG;인하/NNG'], ['인하/NNG;가능성/NNG;낮/VA'], ['인플레이션/NNG;부담/NNG', '가격/NNG;급등/NNG'], ['

In [ ]:
df_total.loc[0,'ngram_word']

"['신용경색/NNG;해소/NNG', '경기/NNG;침체/NNG;피하/VV', '실물/NNG;경기/NNG;침체/NNG', '금리/NNG;하락/NNG', '금리/NNG;우호적/VAX', '위험/NNG;경계/NNG', '신흥국/NNG;신용위험/NNG', '부동산/NNG;위험/NNG', '정책/NNG;위험/NNG', '금리/NNG;인하/NNG', '변동성/NNG;축소/NNG', '금리/NNG;인하/NNG', '인플레이션/NNG;우려/NNG', '금리/NNG;인하/NNG', '유가/NNG;원자재/NNG;가격/NNG;상승/NNG', '경상/NNG;수지/NNG;적자/NNG;확대/NNG', '글로벌/NNG;신용/NNG;불안/NNG', '외국인/NNG;금리/NNG;상승/NNG', '유동성/NNG;여건/NNG;악화/NNG', '원달러/NNG;급등/NNG', 'crs/NNG;금리/NNG;급락/NNG', '금리/NNG;급락/NNG;상승/NNG', '금리/NNG;인하/NNG', '주문/NNG;하락/NNG', '금리/NNG;심하/VA', '금리/NNG;인하/NNG', '인하/NNG;반대/NNG', 'cd/NNG;금리/NNG;상승/NNG', '변동성/NNG;확대/NNG', '금리/NNG;하락/NNG', '하락/NNG;추세/NNG', '붕괴/NNG;위험/NNG', '침체/NNG;피하/VV;없/VA', '부동산/NNG;침체/NNG', '부채/NNG;부담/NNG', '경기/NNG;침체/NNG', '글로벌/NNG;경기/NNG;둔화/NNG', '위험/NNG;없/VA;않/VX', 'gdp/NNG;성장률/NNG;둔화/NNG', '내수/NNG;늘/VV', '부정적/VAX;여파/NNG;우려/NNG', '정책/NNG;대응/NNG;한계/NNG', '본질/NNG;위험/NNG', '금리/NNG;인하/NNG', '인하/NNG;가능성/NNG;낮/VA', '인플레이션/NNG;부담/NNG', '가격/NNG;급등/NNG', '양적완화/NNG;정책/NNG;확대/NNG', '유동성/NNG;압박/NNG', 'fed/NNG

In [ ]:
# call_rate 가져오기
df_cr = df_call_rate[['date', 'RATE', 'change']]
df_cr

,date,RATE,change
0,2005-06-09,3.28,down
1,2005-06-10,3.29,down
2,2005-06-11,3.29,down
3,2005-06-12,3.29,down
4,2005-06-13,3.28,down
...,...,...,...
6372,2022-11-19,3.02,down
6373,2022-11-20,3.02,down
6374,2022-11-21,3.04,down
6375,2022-11-22,3.07,down


In [ ]:
# 날짜별 dict 구성 및 value 입력
date_dict = {}
for idx in df_cr.index :
    date_dict[df_cr.loc[idx, 'date']] = df_cr.loc[idx, 'change']
# date_dict

In [ ]:
# ngram_word 값 없는 것(nan) 처리
drop_list = []
for idx in df_total.index:
    if df_total.loc[idx, 'ngram_word'] == '':
        drop_list.append(idx)
df_total.drop(drop_list, inplace=True)
df_total.isna().sum()

date_prep     0
ngram_prep    0
ngram_word    0
dtype: int64

In [ ]:
df_total

,date_prep,ngram_prep,ngram_word
0,2008-04-01,"[['신용경색/NNG;해소/NNG'], ['경기/NNG;침체/NNG;피하/VV'],...","['신용경색/NNG;해소/NNG', '경기/NNG;침체/NNG;피하/VV', '실물..."
1,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG...","['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG'..."
2,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG...","['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG'..."
3,2008-04-21,"[['경기/NNG;바닥/NNG'], ['인하/NNG;영향/NNG;줄/VV', '인플...","['경기/NNG;바닥/NNG', '인하/NNG;영향/NNG;줄/VV', '인플레이션..."
4,2008-04-28,"[['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격...","['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격/..."
...,...,...,...
284001,2008-05-08,"[['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NN...","['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NNG..."
284002,2008-04-10,"[['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/N...","['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/NN..."
284003,2008-03-20,"[['통화/NNG;안정/NNG'], ['물가/NNG;오름/NNG;확대/NNG', '...","['통화/NNG;안정/NNG', '물가/NNG;오름/NNG;확대/NNG', '물가/..."
284004,2008-03-07,"[['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/N...","['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/NN..."


In [ ]:
# 날짜별 금리 넣어주기
df_total['call_rate'] = df_total['date_prep'].apply(lambda x : date_dict[x])
df_total['call_rate']

0           up
1           up
2           up
3         down
4         down
          ... 
284001    down
284002    down
284003    same
284004      up
284005    same
Name: call_rate, Length: 283944, dtype: object

In [ ]:
df_total

,date_prep,ngram_prep,ngram_word,call_rate
0,2008-04-01,"[['신용경색/NNG;해소/NNG'], ['경기/NNG;침체/NNG;피하/VV'],...","['신용경색/NNG;해소/NNG', '경기/NNG;침체/NNG;피하/VV', '실물...",up
1,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG...","['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG'...",up
2,2008-04-07,"[['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG...","['글로벌/NNG;신용경색/NNG;완화/NNG', '인플레이션/NNG;부담/NNG'...",up
3,2008-04-21,"[['경기/NNG;바닥/NNG'], ['인하/NNG;영향/NNG;줄/VV', '인플...","['경기/NNG;바닥/NNG', '인하/NNG;영향/NNG;줄/VV', '인플레이션...",down
4,2008-04-28,"[['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격...","['fed/NNG;원자재/NNG;가격/NNG;상승/NNG', '원자재/NNG;가격/...",down
...,...,...,...,...
284001,2008-05-08,"[['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NN...","['세계/NNG;경제/NNG;성장/NNG;둔화/NNG', '경기/NNG;상승/NNG...",down
284002,2008-04-10,"[['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/N...","['국제/NNG;유가/NNG;원자재/NNG;가격/NNG;급등/NNG', '금리/NN...",down
284003,2008-03-20,"[['통화/NNG;안정/NNG'], ['물가/NNG;오름/NNG;확대/NNG', '...","['통화/NNG;안정/NNG', '물가/NNG;오름/NNG;확대/NNG', '물가/...",same
284004,2008-03-07,"[['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/N...","['물가/NNG;상승률/NNG;목표/NNG;벗어나/VV', '수요/NNG;압력/NN...",up


In [ ]:
df_total.to_csv('df_total_date_call_rate.csv', index=False)

In [ ]:
# 각 ngram 별
ngram_dict = {}
result_ngram = []
for idx in tqdm(df_total.index):
    result = []
    # 저장된 ngram 복구
    for ngm in re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣,.\/; ]', '', df_total.loc[idx, 'ngram_word']).split(',') :
        temp = [ngm_word.strip() for ngm_word in ngm.split(';')]
        result.append(temp)
    # ngram 조합 복구
    for res in result :
        ngram = ';'.join(res)
        result_ngram.append(ngram)
# set 처리 후 dict 구성
ngram_unique = set([ngram_word for ngram_word in result_ngram])
for nu in ngram_unique :
    ngram_dict[nu] = {'hawkish':0, 'dovish':0}
ngram_dict

  0%|          | 0/283944 [00:00<?, ?it/s]

{'상향/NNG;우세/NNG': {'hawkish': 0, 'dovish': 0},
 '지표/NNG;금리/NNG;오르/VV': {'hawkish': 0, 'dovish': 0},
 '정정불안/NNG;완화/NNG': {'hawkish': 0, 'dovish': 0},
 '소비자/NNG;가격/NNG;상승/NNG': {'hawkish': 0, 'dovish': 0},
 '개선/NNG;내리/VV': {'hawkish': 0, 'dovish': 0},
 '안정/NNG;과도/NNG': {'hawkish': 0, 'dovish': 0},
 '급락/NNG;가능성/NNG;작/VA': {'hawkish': 0, 'dovish': 0},
 '글로벌/NNG;채권시장/NNG;안전자산/NNG;선호/NNG': {'hawkish': 0, 'dovish': 0},
 '낮/VA;성장률/NNG;높/VA': {'hawkish': 0, 'dovish': 0},
 '유가/NNG;급등/NNG;금리/NNG;상승/NNG': {'hawkish': 0, 'dovish': 0},
 '감산/NNG;실패/NNG': {'hawkish': 0, 'dovish': 0},
 '유로존/NNG;지표/NNG;개선/NNG': {'hawkish': 0, 'dovish': 0},
 '매입/NNG;완화/NNG': {'hawkish': 0, 'dovish': 0},
 '자연/NNG;이자율/NNG;하락/NNG': {'hawkish': 0, 'dovish': 0},
 '지표/NNG;되돌/VV': {'hawkish': 0, 'dovish': 0},
 '달러부채/NNG;부담/NNG': {'hawkish': 0, 'dovish': 0},
 '우려/NNG;완화/NNG;채권시장/NNG;강세/NNG': {'hawkish': 0, 'dovish': 0},
 '채권/NNG;금리/NNG;위험/NNG': {'hawkish': 0, 'dovish': 0},
 '높/VA;안정/NNG;성장/NNG': {'hawkish': 0, 'dovish': 0},
 '공포

In [ ]:
df_total.loc[0,'ngram_prep']

In [ ]:
df_total

In [ ]:
# hawkish , dovish 계산
for idx in tqdm(df_total.index):
    ng_list = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣,.\/; ]', '', df_total.loc[idx, 'ngram_word']).split(',')
    change = df_total.loc[idx, 'call_rate']
    for ng in ng_list :
        if ng in ngram_unique:
            if change == 'up' :
                ngram_dict[ng]['hawkish'] += 1
            elif change == 'down' :
                ngram_dict[ng]['dovish'] += 1
            else :
                pass

In [ ]:
df_polarity = pd.DataFrame(ngram_dict).T
df_polarity['cnt'] = df_polarity['hawkish'] + df_polarity['dovish']
df_polarity = df_polarity[df_polarity['cnt'] > 15]
sum_hawkish = df_polarity.hawkish.sum()
sum_dovish = df_polarity.dovish.sum()
df_polarity['prob_hawkish'] = df_polarity['hawkish'] / sum_hawkish
df_polarity['prob_dovish'] = df_polarity['dovish'] / sum_hawkish
df_polarity['polarity_score'] = df_polarity['prob_hawkish'] / df_polarity['prob_dovish']
df_polarity

,hawkish,dovish,cnt,prob_hawkish,prob_dovish,polarity_score
투자심리/NNG;회복/NNG,34,48,82,0.000475,0.000670,0.708333
ecb/NNG;금리/NNG;인하/NNG,54,79,133,0.000754,0.001103,0.683544
대미/NNG;무역/NNG;흑자/NNG,7,10,17,0.000098,0.000140,0.700000
가격/NNG;지정학/NNG;위험/NNG,12,4,16,0.000168,0.000056,3.000000
수입/NNG;중단/NNG,5,14,19,0.000070,0.000196,0.357143
...,...,...,...,...,...,...
인플레이션/NNG;고점/NNG,11,5,16,0.000154,0.000070,2.200000
자본/NNG;통제/NNG,19,16,35,0.000265,0.000223,1.187500
금리/NNG;인상/NNG;지연/NNG,79,76,155,0.001103,0.001062,1.039474
금리/NNG;인하/NNG;기대/NNG;높/VA,3,14,17,0.000042,0.000196,0.214286


In [ ]:
threshold = 1.3
df_polarity_final = df_polarity[(df_polarity['polarity_score'] > threshold) | (df_polarity['polarity_score'] < 1/threshold)]
df_polarity_final

In [ ]:
# 그레이 에리어 검증
df_polarity_final[(df_polarity_final['polarity_score'] > 1/ threshold) & (df_polarity_final['polarity_score'] < threshold)]

In [ ]:
df_polarity_final = df_polarity_final.reset_index()
df_polarity_final

In [ ]:
df_polarity_final.to_csv('polarity_score_final.csv', index=False)

In [ ]:
pd.read_csv('polarity_score_final.csv')

,index,hawkish,dovish,cnt,prob_hawkish,prob_dovish,polarity_score
0,투자심리/NNG;회복/NNG,34,48,82,0.000475,0.000670,0.708333
1,ecb/NNG;금리/NNG;인하/NNG,54,79,133,0.000754,0.001103,0.683544
2,대미/NNG;무역/NNG;흑자/NNG,7,10,17,0.000098,0.000140,0.700000
3,가격/NNG;지정학/NNG;위험/NNG,12,4,16,0.000168,0.000056,3.000000
4,수입/NNG;중단/NNG,5,14,19,0.000070,0.000196,0.357143
...,...,...,...,...,...,...,...
1324,수출/NNG;제한/NNG,7,10,17,0.000098,0.000140,0.700000
1325,비용/NNG;늘/VV,12,8,20,0.000168,0.000112,1.500000
1326,인플레이션/NNG;고점/NNG,11,5,16,0.000154,0.000070,2.200000
1327,금리/NNG;인하/NNG;기대/NNG;높/VA,3,14,17,0.000042,0.000196,0.214286


,index,hawkish,dovish,cnt,prob_hawkish,prob_dovish,polarity_score
0,투자심리/NNG;회복/NNG,34,48,82,0.000475,0.000670,0.708333
1,ecb/NNG;금리/NNG;인하/NNG,54,79,133,0.000754,0.001103,0.683544
2,대미/NNG;무역/NNG;흑자/NNG,7,10,17,0.000098,0.000140,0.700000
3,가격/NNG;지정학/NNG;위험/NNG,12,4,16,0.000168,0.000056,3.000000
4,수입/NNG;중단/NNG,5,14,19,0.000070,0.000196,0.357143
...,...,...,...,...,...,...,...
1324,수출/NNG;제한/NNG,7,10,17,0.000098,0.000140,0.700000
1325,비용/NNG;늘/VV,12,8,20,0.000168,0.000112,1.500000
1326,인플레이션/NNG;고점/NNG,11,5,16,0.000154,0.000070,2.200000
1327,금리/NNG;인하/NNG;기대/NNG;높/VA,3,14,17,0.000042,0.000196,0.214286
